In [1]:
%matplotlib inline
import numpy as np
import scipy
import matplotlib.pyplot as plt
import IPython.display
import essentia
import essentia.standard as es
import librosa
import librosa.display
from collections import Counter
from utils import *

In [6]:
for file in os.listdir('sounds/'):
    beats = []
    beats_list = []
    onsets = []
    v = []
    sum_beats = []
    drift = []
    drift2 = []
    xlabel = []
    
    if '.wav' not in file: continue
    filename = file
    load_file = 'sounds/' + filename  
    
    # load audio 
    loader = es.MonoLoader(filename=load_file)
    audio = loader()

    # read onsets
    read_onset = 'onsets/' + 'onset.' + filename[:-3] + 'csv'
    onsets = np.genfromtxt(read_onset)

    # find beats
    bt = es.BeatTrackerMultiFeature()
    beats, _ = bt(audio)

    # set of interonset intervals
    beats_list = np.ndarray.tolist(beats)
    v = [beats_list[i+1]-beats_list[i] for i in range(len(beats_list)-1)]

    # calculate drift
    sum_beats = np.cumsum(v)
    rho = np.mean(v)

    for i in range(len(sum_beats)):
        a = sum_beats[i] - i * rho
        drift.append(a)

    # calculate drift with reference to 0 --- mean of drift a good idea?    
    #for i in range(len(drift)):
    #    b = drift[i] - np.mean(drift)
    #    drift2.append(b)
        
    #xticks
    sum_beats = np.cumsum(v)
    
    for i in range(len(sum_beats)):
        c = beats[i]  #is this correct????
        xlabel.append(c)
        
    # plot
    plt.plot(xlabel, drift)
    plt.ylim(0,1)
    plt.xlabel('seconds')
    plt.ylabel('drift')
    title = 'drummer1.' + filename[:-4]
    plt.title(title)
    
    # save plot to folder 'drift'
    figure = 'drift/drift_withoutreference_01/' + title +'.jpg'
    plt.savefig(figure)
    plt.close()

In [36]:
''''
# rhythm extractor
avg_bpm, beat_start, confidence, tempo, beat_duration = es.RhythmExtractor2013(method='multifeature')(audio)
beats2_list = np.ndarray.tolist(beat_start)
t = [abs(beats2_list[i+1]-beats2_list[i]) for i in range(len(beats2_list)-1)]